In [ ]:
using BilevelJuMP
using MibS_jll


t = [
    8 7 6 9 1 2 5 3 4
    9 5 4 3 8 7 6 1 2
    2 1 3 6 4 5 7 8 9
    7 3 8 4 5 6 2 9 1
    5 6 1 8 2 9 3 4 7
    4 2 9 7 3 1 8 6 5
    6 4 2 1 7 8 9 5 3
    3 9 5 2 6 4 1 7 8
    1 8 7 5 9 3 4 2 6
    ]

n = 4
model = BilevelModel()
@variable(Upper(model), y[1:n, 1:n], Bin, start = 1)
@variable(Lower(model), x[1:n, 1:n, 1:9], Bin)
@variable(Lower(model), z, Bin)

@objective(Upper(model), Min, sum(y[i, j] for i in 1:n, j in 1:n))
@constraint(Upper(model), z >=1)



@objective(Lower(model), Min, z)


for i in 1:n  # For each row
    for j in 1:n  # and each column
        # Sum across all the possible digits. One and only one of the digits
        # can be in this cell, so the sum must be equal to one.
        @constraint(Lower(model), sum(x[i, j, k] for k in 1:n) <= 1)
        @constraint(Lower(model), sum(x[i, j, k] for k in 1:n) >= 1)
    end
end

for i in 1:n
    for k in 1:n
        @constraint(Lower(model), sum(x[ind, j, k] for j in 1:n) >= 1)
        @constraint(Lower(model), sum(x[ind, j, k] for j in 1:n) <= 1)
    end
end

for j in 1:n
    for k in 1:n
        @constraint(Lower(model), sum(x[i, ind, k] for i in 1:n) <= 1)
        @constraint(Lower(model), sum(x[i, ind, k] for i in 1:n) >= 1)
    end
end

for i in 1:2:3
    for j in 1:2:3
        for k in 1:n
            @constraint(Lower(model), sum(x[r, c, k] for r in i:(i+1), c in j:(j+1)) <= 1)
            @constraint(Lower(model), sum(x[r, c, k] for r in i:(i+1), c in j:(j+1)) >= 1)
        end
    end
end

for i in 1:n
    for j in 1:n
            @constraint(Lower(model), x[i, j, G[i,j]] >= y[i, j])
    end
end

@constraint(Lower(model), sum(x[i, j, G[i,j]] for i in 1:n, j in 1:n) - z <= n^2 - 1)

# Solve with MibS
solution = BilevelJuMP.solve_with_MibS(model, MibS_jll.mibs)

